In [1]:
import os
import pandas as pd
from enum import unique
import datetime
from XdbSearchIP import iptest
import openpyxl
from docx import Document
from docx.shared import Pt


In [2]:
################################################ 定义变量和函数 ######################################################
# 定义日报工作目录路径变量
ribaopath = 'C:\\Users\\11298\\Desktop\\日报\\'
xzpath = 'C:\\Users\\11298\\Downloads\\'

# 定义查询ip归属地的函数
def ip2r(ip):
    r_c = iptest.searchWithFile(ip=ip).split('|')[0]
    r_p = iptest.searchWithFile(ip=ip).split('|')[2]
    if r_c == '中国':
        return r_p
    else:
        return r_c
    
# 定义需要用到的日期变量
now = datetime.datetime.now()
riqi = str(now.year)+'-'+str(now.month)+'-'+str(now.day)
onlyriqi = str(now.month)+'月'+str(now.day)+'日'

In [3]:
########################################### 读取“alarm_log_xxxxxxxxxxxx.csv”文件，得到日志df ############################################
# 通过读取态势日志csv,创建态势日志df
sa_df = pd.read_csv(xzpath+'alarm_log_1715500840871.csv')

In [4]:
################################################ 读取“风险log.xlsx”文件，得到资产df ######################################################
#读取风险log.xlsx工作簿,返回类型为集合，每个sheet是一个df，是集合的一个元素
fengxl = pd.read_excel(ribaopath+'风险log.xlsx',sheet_name=None)

# 从工作簿集合中提取'资产'sheet,返回类型为dataframe
asset = fengxl.get('资产')
# 将'ip'列设置为行索引
asset_idxip = asset.set_index('ip')

# 从工作簿集合中提取'log_云外异常'sheet,返回类型为dataframe
log_ywycdf = fengxl.get('log_云外异常')

# 从工作簿集合中提取'log_本地异常'sheet,返回类型为dataframe
log_bdycdf = fengxl.get('log_本地异常')


In [5]:
#################################################### 对从原始告警日志df进行预处理 ########################################################
# 对‘源IP：端口’列分列，并设置列名称
sasip_fl = sa_df['源IP:端口'].str.split(':',expand=True)
sasip_fl.columns = ['sip','源端口']

# 对‘目的IP：端口’列分列，并设置列名称
sadip_fl = sa_df['目的IP:端口'].str.split(':',expand=True)
sadip_fl.columns = ['dip','目的端口']

# 拼接
sadf_splited = pd.concat([sa_df,sasip_fl,sadip_fl],axis=1)

# 删除事件告警中无用信息，缩短
sadf_splited['事件告警'] = sadf_splited['事件告警'].str.replace('防火墙-高危-','')

In [6]:
################################################ 整理统计数据 ####################################################
# 载荷投递数量
sadf_fztj = sadf_splited.groupby('事件告警').size()
if '异常包攻击' in sadf_fztj:
    zhtd_num = sadf_fztj['异常包攻击']
else:
    zhtd_num = 0

# 侦察跟踪数量
zcgz_num = sadf_splited.shape[0]-zhtd_num

In [7]:
################################################ 计算告警峰值时间 ####################################################
# 转换‘开始时间’列为pandas的datetime类型，并设置为索引
sadf_qs = sadf_splited.set_index(pd.to_datetime(sadf_splited['开始时间']))
# 以小时为单位重取样resample，倒序排序，取top1
sadf_top1ts = sadf_qs.sort_index().resample('H').size().sort_values(ascending=False).index[0]

In [8]:
################################################ 态势告警日志分析-源ip #################################################
# 按sip分组，对‘事件告警’聚合，得到聚合series，将series转换为df
sadf_sipgrouped = sadf_splited.groupby('sip')
sadf_sipjh = sadf_sipgrouped['事件告警'].agg('unique')
sadf_sipjh_df = pd.DataFrame(sadf_sipjh)
sadf_sipjh_df

,事件告警
sip,
1.14.28.124,[异常扫描]
103.122.177.153,[域外攻击源-命令执行]
103.142.208.170,"[域外攻击源-通用攻击, 域外攻击源-SQL注入, 域外攻击源-SQL注入攻击]"
103.149.26.230,[域外攻击源-请求访问]
103.68.195.46,[域外攻击源-命令执行]
...,...
83.147.52.37,"[域外攻击源-命令执行, 域外攻击源-Web攻击, 异常扫描]"
91.92.247.105,[域外攻击源-Web攻击]
91.92.247.109,[域外攻击源-Web攻击]


In [9]:
# 得到每个攻击源的告警数量series，并追加到sadf_sipjh_df
sadf_sipjh_df['告警数量'] = sadf_sipgrouped.size()
sadf_sipjh_df

,事件告警,告警数量
sip,,
1.14.28.124,[异常扫描],2
103.122.177.153,[域外攻击源-命令执行],1
103.142.208.170,"[域外攻击源-通用攻击, 域外攻击源-SQL注入, 域外攻击源-SQL注入攻击]",5
103.149.26.230,[域外攻击源-请求访问],1
103.68.195.46,[域外攻击源-命令执行],1
...,...,...
83.147.52.37,"[域外攻击源-命令执行, 域外攻击源-Web攻击, 异常扫描]",42
91.92.247.105,[域外攻击源-Web攻击],18
91.92.247.109,[域外攻击源-Web攻击],6


In [10]:
# 将索引作为普通列插入到第一列
sadf_sipjh_df.insert(0,'sip',sadf_sipjh_df.index)


In [11]:
# 修改列名'事件告警'为'告警类型'
sadf_sipjh_df.columns = ['sip','告警类型','告警数量']
# 添加3个新列，'归属地'、'风险等级'、'操作建议'
sadfsip_convnon = sadf_sipjh_df.reindex(columns=['sip','告警类型','告警数量','归属地','风险等级','操作建议'])

# 查询攻击源ip的归属地，并填充‘归属地’列
sadfsip_convnon['归属地'] = sadfsip_convnon['sip'].apply(lambda x : ip2r(x))


In [12]:
# 筛选出告警数量大于等于10的行
safxsip_num = sadfsip_convnon[sadfsip_convnon['告警数量']>=10]
# 筛选出告警类型大于等于3种的行
safxsip_lx = sadfsip_convnon[sadfsip_convnon['告警类型'].apply(len)>=3]


In [13]:
# 拼接
safxsip_convnon = pd.concat([safxsip_num,safxsip_lx],axis=0)
# 去重
safxsip_convnon.drop_duplicates(subset='sip',inplace=True)
safxsip_convnon

,sip,告警类型,告警数量,归属地,风险等级,操作建议
sip,,,,,,
141.98.11.18,141.98.11.18,"[域外攻击源-通用攻击, 域外攻击源-用户提权, 异常扫描]",10,立陶宛,NaN,NaN
159.203.73.253,159.203.73.253,"[域外攻击源-用户提权, 异常扫描, 域外攻击源-通用攻击]",15,美国,NaN,NaN
27.129.145.95,27.129.145.95,[防火墙-用户提权],13,河北省,NaN,NaN
35.203.211.16,35.203.211.16,"[域外攻击源-漏洞扫描, 异常扫描]",10,英国,NaN,NaN
39.100.79.68,39.100.79.68,[异常扫描],24,河北省,NaN,NaN
77.91.87.196,77.91.87.196,"[域外攻击源-命令执行, 异常扫描, 域外攻击源-服务器防护, 域外攻击源-用户提权]",49,俄罗斯,NaN,NaN
83.147.52.37,83.147.52.37,"[域外攻击源-命令执行, 域外攻击源-Web攻击, 异常扫描]",42,西班牙,NaN,NaN
91.92.247.105,91.92.247.105,[域外攻击源-Web攻击],18,保加利亚,NaN,NaN
94.156.65.66,94.156.65.66,[域外攻击源-Web攻击],18,保加利亚,NaN,NaN


In [14]:
# 风险df的行数
sip_rownum = safxsip_convnon.shape[0]
# 分析整理得到“公网异常ip情况”df
for j in range(0,sip_rownum):
    
    alertnum = safxsip_convnon.loc[:,'告警数量'][j]
    alerttype = len(safxsip_convnon.loc[:,'告警类型'][j])
    alert_list_str = ''.join(list(safxsip_convnon.iloc[j,1]))

    if '域外攻击源' in alert_list_str:
        safxsip_convnon.iloc[j,4] = '高'
        safxsip_convnon.iloc[j,5] = '封禁'
    elif '恶意源ip' in alert_list_str:
        safxsip_convnon.iloc[j,4] = '高'
        safxsip_convnon.iloc[j,5] = '封禁'
    elif '异常扫描' in alert_list_str:
        safxsip_convnon.iloc[j,4] = '高'
        safxsip_convnon.iloc[j,5] = '封禁'
    elif alertnum >= 100 or alerttype >= 4:
        safxsip_convnon.iloc[j,4] = '高'
        safxsip_convnon.iloc[j,5] = '封禁'
    elif 30 <= alertnum < 100:
        safxsip_convnon.iloc[j,4] = '中'
        safxsip_convnon.iloc[j,5] = '观察'
    elif 10 <= alertnum < 30 and 2 <= alerttype <= 3:
        safxsip_convnon.iloc[j,4] = '中'
        safxsip_convnon.iloc[j,5] = '观察'
    else:
        safxsip_convnon.iloc[j,4] = '低'
        safxsip_convnon.iloc[j,5] = '观察'
# 按告警数量倒序排序
safxsip_toword = safxsip_convnon.sort_values(by=['告警数量'],ascending=[False])
# 插入序号列
safxsip_toword.insert(0,'序号',range(1,sip_rownum+1))
# 告警类型转换为字符串，并添加换行符
for e in range(0,safxsip_toword.shape[0]):
    safxsip_toword['告警类型'][e] = '\n'.join(safxsip_toword['告警类型'][e])
safxsip_toword

C:\Users\11298\AppData\Local\Temp\ipykernel_18560\1421287257.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  safxsip_toword['告警类型'][e] = '\n'.join(safxsip_toword['告警类型'][e])


,序号,sip,告警类型,告警数量,归属地,风险等级,操作建议
sip,,,,,,,
77.91.87.196,1,77.91.87.196,域外攻击源-命令执行\n异常扫描\n域外攻击源-服务器防护\n域外攻击源-用户提权,49,俄罗斯,高,封禁
83.147.52.37,2,83.147.52.37,域外攻击源-命令执行\n域外攻击源-Web攻击\n异常扫描,42,西班牙,高,封禁
39.100.79.68,3,39.100.79.68,异常扫描,24,河北省,高,封禁
91.92.247.105,4,91.92.247.105,域外攻击源-Web攻击,18,保加利亚,高,封禁
94.156.65.66,5,94.156.65.66,域外攻击源-Web攻击,18,保加利亚,高,封禁
159.203.73.253,6,159.203.73.253,域外攻击源-用户提权\n异常扫描\n域外攻击源-通用攻击,15,美国,高,封禁
27.129.145.95,7,27.129.145.95,防火墙-用户提权,13,河北省,低,观察
141.98.11.18,8,141.98.11.18,域外攻击源-通用攻击\n域外攻击源-用户提权\n异常扫描,10,立陶宛,高,封禁
35.203.211.16,9,35.203.211.16,域外攻击源-漏洞扫描\n异常扫描,10,英国,高,封禁


In [15]:
# 整理得到“封禁ip”df
safxsip_fj = safxsip_toword[['sip','归属地']][safxsip_toword['操作建议']=='封禁']
safxsip_fj.insert(0,'序号',range(1,safxsip_fj.shape[0]+1))
safxsip_fj[['封禁方','封禁原因']] = ['租户侧','高危']
safxsip_fj

,序号,sip,归属地,封禁方,封禁原因
sip,,,,,
77.91.87.196,1,77.91.87.196,俄罗斯,租户侧,高危
83.147.52.37,2,83.147.52.37,西班牙,租户侧,高危
39.100.79.68,3,39.100.79.68,河北省,租户侧,高危
91.92.247.105,4,91.92.247.105,保加利亚,租户侧,高危
94.156.65.66,5,94.156.65.66,保加利亚,租户侧,高危
159.203.73.253,6,159.203.73.253,美国,租户侧,高危
141.98.11.18,7,141.98.11.18,立陶宛,租户侧,高危
35.203.211.16,8,35.203.211.16,英国,租户侧,高危
103.142.208.170,9,103.142.208.170,柬埔寨,租户侧,高危


In [16]:
# 整理得到写入风险log.xlsx的公网异常df
safxsip_toxlsx = safxsip_toword.loc[:,['sip','告警类型','告警数量','归属地']]
safxsip_toxlsx.insert(0,'日期',riqi)
safxsip_toxlsx

,日期,sip,告警类型,告警数量,归属地
sip,,,,,
77.91.87.196,2024-5-12,77.91.87.196,域外攻击源-命令执行\n异常扫描\n域外攻击源-服务器防护\n域外攻击源-用户提权,49,俄罗斯
83.147.52.37,2024-5-12,83.147.52.37,域外攻击源-命令执行\n域外攻击源-Web攻击\n异常扫描,42,西班牙
39.100.79.68,2024-5-12,39.100.79.68,异常扫描,24,河北省
91.92.247.105,2024-5-12,91.92.247.105,域外攻击源-Web攻击,18,保加利亚
94.156.65.66,2024-5-12,94.156.65.66,域外攻击源-Web攻击,18,保加利亚
159.203.73.253,2024-5-12,159.203.73.253,域外攻击源-用户提权\n异常扫描\n域外攻击源-通用攻击,15,美国
27.129.145.95,2024-5-12,27.129.145.95,防火墙-用户提权,13,河北省
141.98.11.18,2024-5-12,141.98.11.18,域外攻击源-通用攻击\n域外攻击源-用户提权\n异常扫描,10,立陶宛
35.203.211.16,2024-5-12,35.203.211.16,域外攻击源-漏洞扫描\n异常扫描,10,英国


In [17]:
################################################ 态势告警日志分析-目的ip ###########################################################
# 按dip分组，对‘事件告警’聚合
sadf_dipgrouped = sadf_splited.groupby('dip')
sadf_dipjhs = sadf_dipgrouped['事件告警'].agg('unique')
sadf_dipjh_df = pd.DataFrame(sadf_dipjhs)
sadf_dipjh_df

,事件告警
dip,
103.142.208.170,[异常外联]
111.7.100.22,[异常外联]
144.7.110.1,"[域外攻击源-命令执行, 域外攻击源-服务器防护, 域外攻击源-Web攻击, 异常扫描, 域..."
144.7.110.10,[域外攻击源-漏洞扫描]
144.7.110.111,[域外攻击源-漏洞扫描]
...,...
203.110.222.96,"[域外攻击源-漏洞扫描, 域外攻击源-请求访问]"
203.110.222.99,[异常扫描]
203.34.106.28,[异常外联]


In [18]:
# 将目的ip的告警数量追加至聚合df的‘告警数量’列
sadf_dipjh_df['告警数量'] = sadf_dipgrouped.size()
# 将索引列作为普通列插入至第1列
sadf_dipjh_df.insert(0,'dip',sadf_dipjh_df.index)


In [19]:
# sadf_dipjh_df = sadf_dipjh_df.reindex(columns=['dip','事件告警','告警数量','系统','风险等级','操作建议'])

In [20]:

# 利用布尔列表筛选出告警数量大于50的行
sadf_dip_fx_num = sadf_dipjh_df[sadf_dipjh_df['告警数量']>50]
# 筛选出告警类型数量大于3种的行
sadf_dip_fx_lx = sadf_dipjh_df[sadf_dipjh_df['事件告警'].apply(len)>3]
# 拼接
safxdip = pd.concat([sadf_dip_fx_num,sadf_dip_fx_lx],axis=0)
# 去重
safxdip.drop_duplicates(subset='dip',inplace=True)
safxdip

,dip,事件告警,告警数量
dip,,,
144.7.110.1,144.7.110.1,"[域外攻击源-命令执行, 域外攻击源-服务器防护, 域外攻击源-Web攻击, 异常扫描, 域...",32
144.7.110.134,144.7.110.134,"[域外攻击源-命令执行, 域外攻击源-漏洞扫描, 域外攻击源-用户提权, 异常扫描, 域外攻...",24
144.7.110.184,144.7.110.184,"[防火墙-用户提权, 域外攻击源-请求访问, 域外攻击源-用户提权, 域外攻击源-通用攻击]",7
144.7.111.16,144.7.111.16,"[防火墙-通用攻击, 防火墙-服务器防护, 防火墙-目录遍历, 异常扫描, 域外攻击源-请求访问]",17
144.7.111.31,144.7.111.31,"[域外攻击源-命令执行, 域外攻击源-服务器防护, 域外攻击源-Web攻击, 域外攻击源-请...",30
144.7.111.36,144.7.111.36,"[域外攻击源-用户提权, 域外攻击源-命令执行, 异常包攻击, 异常扫描, 域外攻击源-安全...",13
144.7.111.92,144.7.111.92,"[域外攻击源-命令执行, 异常扫描, 域外攻击源-通用攻击, 域外攻击源-Web攻击, 域外...",18
203.110.221.146,203.110.221.146,"[域外攻击源-请求访问, 域外攻击源-漏洞扫描, 防火墙-用户提权, 域外攻击源-用户提权]",4
203.110.221.27,203.110.221.27,"[域外攻击源-请求访问, 域外攻击源-命令执行, 域外攻击源-Web攻击, 域外攻击源-通用...",38


In [21]:
# 获取风险目的ip对应的资产信息
safxdip_zc = asset_idxip.loc[list(safxdip.index)]
# 拼接
safxdip_convnon = pd.concat([safxdip,safxdip_zc],axis=1)
# 追加‘风险等级’和‘操作建议’列
safxdip_convnon = safxdip_convnon.reindex(columns=['dip','事件告警','告警数量','部门','系统','风险等级','操作建议'])
# 修改列名‘事件告警’为‘事件类型’
safxdip_convnon.rename(columns={'事件告警':'事件类型'},inplace=True)
safxdip_convnon

,dip,事件类型,告警数量,部门,系统,风险等级,操作建议
144.7.110.1,144.7.110.1,"[域外攻击源-命令执行, 域外攻击源-服务器防护, 域外攻击源-Web攻击, 异常扫描, 域...",32,雄安集团,雄安集团官方网站系统,NaN,NaN
144.7.110.134,144.7.110.134,"[域外攻击源-命令执行, 域外攻击源-漏洞扫描, 域外攻击源-用户提权, 异常扫描, 域外攻...",24,公服局,互联网+政务服务平台-建行,NaN,NaN
144.7.110.184,144.7.110.184,"[防火墙-用户提权, 域外攻击源-请求访问, 域外攻击源-用户提权, 域外攻击源-通用攻击]",7,雄安集团,数字孪生课题系统-软通,NaN,NaN
144.7.111.16,144.7.111.16,"[防火墙-通用攻击, 防火墙-服务器防护, 防火墙-目录遍历, 异常扫描, 域外攻击源-请求访问]",17,改发局,国有资产监管系统,NaN,NaN
144.7.111.31,144.7.111.31,"[域外攻击源-命令执行, 域外攻击源-服务器防护, 域外攻击源-Web攻击, 域外攻击源-请...",30,公服局,互联网+政务服务平台-建行,NaN,NaN
144.7.111.36,144.7.111.36,"[域外攻击源-用户提权, 域外攻击源-命令执行, 异常包攻击, 异常扫描, 域外攻击源-安全...",13,雄安集团,雄安集团块数据平台管理节点,NaN,NaN
144.7.111.92,144.7.111.92,"[域外攻击源-命令执行, 异常扫描, 域外攻击源-通用攻击, 域外攻击源-Web攻击, 域外...",18,公服局,数字信用信息平台,NaN,NaN
203.110.221.146,203.110.221.146,"[域外攻击源-请求访问, 域外攻击源-漏洞扫描, 防火墙-用户提权, 域外攻击源-用户提权]",4,规建局,岩土地质信息业务应用系统,NaN,NaN
203.110.221.27,203.110.221.27,"[域外攻击源-请求访问, 域外攻击源-命令执行, 域外攻击源-Web攻击, 域外攻击源-通用...",38,雄安集团,雄安集团区块链招采平台,NaN,NaN
203.110.221.72,203.110.221.72,"[域外攻击源-Web攻击, 域外攻击源-请求访问, 域外攻击源-命令执行, 域外攻击源-信息...",25,党群工作部,雄安新区信访信息系统,NaN,NaN


In [22]:
safxdip_convnon.iloc[0,4]

'雄安集团官方网站系统'

In [23]:
# 行数
dip_rownum = safxdip_convnon.shape[0]
# 填充‘风险等级’和‘操作建议’列
for i in range(0,dip_rownum):
    if len(safxdip_convnon.iloc[i,1])>3 and safxdip_convnon.iloc[i,2] >= 100:
        safxdip_convnon.iloc[i,5] = '高'
        safxdip_convnon.iloc[i,6] = '通知用户排查风险和系统加固'
    elif len(safxdip_convnon.iloc[i,1])>3 or safxdip_convnon.iloc[i,2] >= 100:
        safxdip_convnon.iloc[i,5] = '中'
        safxdip_convnon.iloc[i,6] = '通知用户排查风险和系统加固'
    else:
        safxdip_convnon.iloc[i,5] = '低'
        safxdip_convnon.iloc[i,6] = '通知用户排查风险和系统加固'
# word表格不需要‘部门’列       
safxdip_toword = safxdip_convnon.loc[:,['dip','事件类型','告警数量','系统','风险等级','操作建议']]
# 排序
safxdip_toword = safxdip_toword.sort_values(by=['告警数量'],ascending=[False])
# 插入序号
safxdip_toword.insert(0,'序号',range(1,len(safxdip_toword.index)+1))
# ‘事件类型’列转换为字符串
for e in range(0,safxdip_toword.shape[0]):
    safxdip_toword['事件类型'][e] = '\n'.join(safxdip_toword['事件类型'][e])
safxdip_toword

C:\Users\11298\AppData\Local\Temp\ipykernel_18560\3238397055.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  safxdip_toword['事件类型'][e] = '\n'.join(safxdip_toword['事件类型'][e])


,序号,dip,事件类型,告警数量,系统,风险等级,操作建议
203.110.221.27,1,203.110.221.27,域外攻击源-请求访问\n域外攻击源-命令执行\n域外攻击源-Web攻击\n域外攻击源-通用攻...,38,雄安集团区块链招采平台,中,通知用户排查风险和系统加固
144.7.110.1,2,144.7.110.1,域外攻击源-命令执行\n域外攻击源-服务器防护\n域外攻击源-Web攻击\n异常扫描\n域外...,32,雄安集团官方网站系统,中,通知用户排查风险和系统加固
144.7.111.31,3,144.7.111.31,域外攻击源-命令执行\n域外攻击源-服务器防护\n域外攻击源-Web攻击\n域外攻击源-请求...,30,互联网+政务服务平台-建行,中,通知用户排查风险和系统加固
203.110.221.72,4,203.110.221.72,域外攻击源-Web攻击\n域外攻击源-请求访问\n域外攻击源-命令执行\n域外攻击源-信息泄...,25,雄安新区信访信息系统,中,通知用户排查风险和系统加固
144.7.110.134,5,144.7.110.134,域外攻击源-命令执行\n域外攻击源-漏洞扫描\n域外攻击源-用户提权\n异常扫描\n域外攻击...,24,互联网+政务服务平台-建行,中,通知用户排查风险和系统加固
203.110.222.102,6,203.110.222.102,域外攻击源-请求访问\n异常包攻击\n异常扫描\n域外攻击源-用户提权\n域外攻击源-漏洞扫描,19,雄安车位管理系统,中,通知用户排查风险和系统加固
144.7.111.92,7,144.7.111.92,域外攻击源-命令执行\n异常扫描\n域外攻击源-通用攻击\n域外攻击源-Web攻击\n域外攻...,18,数字信用信息平台,中,通知用户排查风险和系统加固
144.7.111.16,8,144.7.111.16,防火墙-通用攻击\n防火墙-服务器防护\n防火墙-目录遍历\n异常扫描\n域外攻击源-请求访问,17,国有资产监管系统,中,通知用户排查风险和系统加固
144.7.111.36,9,144.7.111.36,域外攻击源-用户提权\n域外攻击源-命令执行\n异常包攻击\n异常扫描\n域外攻击源-安全绕...,13,雄安集团块数据平台管理节点,中,通知用户排查风险和系统加固
144.7.110.184,10,144.7.110.184,防火墙-用户提权\n域外攻击源-请求访问\n域外攻击源-用户提权\n域外攻击源-通用攻击,7,数字孪生课题系统-软通,中,通知用户排查风险和系统加固


In [24]:
# 整理得到写入风险logs.xlsx的目的地址df
safxdip_toxlsx = safxdip_convnon[['dip','事件类型','告警数量','部门','系统']]
safxdip_toxlsx.insert(0,column='日期',value=riqi)
safxdip_toxlsx = safxdip_toxlsx.sort_values(by=['告警数量'],ascending=[False])
for e in range(0,safxdip_toxlsx.shape[0]):
    safxdip_toxlsx['事件类型'][e] = '\n'.join(safxdip_toxlsx['事件类型'][e])
safxdip_toxlsx

C:\Users\11298\AppData\Local\Temp\ipykernel_18560\1002295267.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  safxdip_toxlsx['事件类型'][e] = '\n'.join(safxdip_toxlsx['事件类型'][e])


,日期,dip,事件类型,告警数量,部门,系统
203.110.221.27,2024-5-12,203.110.221.27,域外攻击源-请求访问\n域外攻击源-命令执行\n域外攻击源-Web攻击\n域外攻击源-通用攻...,38,雄安集团,雄安集团区块链招采平台
144.7.110.1,2024-5-12,144.7.110.1,域外攻击源-命令执行\n域外攻击源-服务器防护\n域外攻击源-Web攻击\n异常扫描\n域外...,32,雄安集团,雄安集团官方网站系统
144.7.111.31,2024-5-12,144.7.111.31,域外攻击源-命令执行\n域外攻击源-服务器防护\n域外攻击源-Web攻击\n域外攻击源-请求...,30,公服局,互联网+政务服务平台-建行
203.110.221.72,2024-5-12,203.110.221.72,域外攻击源-Web攻击\n域外攻击源-请求访问\n域外攻击源-命令执行\n域外攻击源-信息泄...,25,党群工作部,雄安新区信访信息系统
144.7.110.134,2024-5-12,144.7.110.134,域外攻击源-命令执行\n域外攻击源-漏洞扫描\n域外攻击源-用户提权\n异常扫描\n域外攻击...,24,公服局,互联网+政务服务平台-建行
203.110.222.102,2024-5-12,203.110.222.102,域外攻击源-请求访问\n异常包攻击\n异常扫描\n域外攻击源-用户提权\n域外攻击源-漏洞扫描,19,雄安集团,雄安车位管理系统
144.7.111.92,2024-5-12,144.7.111.92,域外攻击源-命令执行\n异常扫描\n域外攻击源-通用攻击\n域外攻击源-Web攻击\n域外攻...,18,公服局,数字信用信息平台
144.7.111.16,2024-5-12,144.7.111.16,防火墙-通用攻击\n防火墙-服务器防护\n防火墙-目录遍历\n异常扫描\n域外攻击源-请求访问,17,改发局,国有资产监管系统
144.7.111.36,2024-5-12,144.7.111.36,域外攻击源-用户提权\n域外攻击源-命令执行\n异常包攻击\n异常扫描\n域外攻击源-安全绕...,13,雄安集团,雄安集团块数据平台管理节点
144.7.110.184,2024-5-12,144.7.110.184,防火墙-用户提权\n域外攻击源-请求访问\n域外攻击源-用户提权\n域外攻击源-通用攻击,7,雄安集团,数字孪生课题系统-软通


In [25]:
# 整理得到“分部门统计本地异常ip”df
safxdip_bms = safxdip_toxlsx.groupby('部门').size()
safxdip_bm = safxdip_bms.to_frame(name='异常IP数量')
safxdip_bm.reset_index(inplace=True)
safxdip_bm.insert(0,'序号',range(1,safxdip_bm.shape[0]+1))
safxdip_bm

,序号,部门,异常IP数量
0,1,党群工作部,1
1,2,公服局,3
2,3,改发局,1
3,4,规建局,1
4,5,雄安集团,5


In [26]:
############################################## 利用pthon-docx复制日报模板并生成新日报 #############################################################
# 定义各种所需变量和函数
docxd = Document(ribaopath+'租户侧安全日报_模板.docx')
t0 = docxd.tables[0]
t1 = docxd.tables[1]
t2 = docxd.tables[2]
t3 = docxd.tables[3]
# 定义将df填入相应table的函数
def df2wd(tn,towdf):
    dfrn = towdf.shape[0]
    dfcn = towdf.shape[1]
    for i in range(0,dfrn):
        tn.add_row()
        for j in range(0,dfcn):
            tn.cell(i+1,j).text = str(towdf.iloc[i,j])

# 填充文字描述部分
docxd.paragraphs[1].text = riqi
docxd.paragraphs[4].add_run(str(sadf_splited.shape[0])+'条告警 ，其中侦察跟踪'+str(zcgz_num)+'条，载荷投递'+str(zhtd_num)+'条，其余均为0条，异常流量均被阻断。')
docxd.paragraphs[5].add_run(str(sadf_top1ts.day)+'日'+str(sadf_top1ts.hour)+'时')
docxd.paragraphs[6].add_run(str(safxdip_toword.shape[0])+'个，如下：')
docxd.paragraphs[8].add_run(str(safxsip_toword.shape[0])+'个，结合威胁情报分析，已针对IP风险等级进行处置分类，详情参看附属表格。')

# 将准备好的各df写入对应table
df2wd(t0,safxdip_bm)
df2wd(t1,safxsip_toword)
df2wd(t2,safxsip_fj)
df2wd(t3,safxdip_toword)

# 修改表格字体大小
for t in [t0,t1,t2,t3]:
    for row in t.rows:
        for cell in row.cells:
            paragraphs = cell.paragraphs
            for paragraph in paragraphs:
                for run in paragraph.runs:
                    font = run.font
                    font.size = Pt(11)


# 保存
docxd.save(ribaopath+'租户侧安全日报_'+onlyriqi+'.docx')

In [27]:
################################################# 追加写入风险log.xlsx文件############################################################
# 读取待写入excel
fxwb = openpyxl.load_workbook(ribaopath+'风险log.xlsx')

In [ ]:
# 定义写入函数
def toexlsx(df,sht):
    st = fxwb[sht]
    sht_rn = st.max_row
    for i in range(0,df.shape[0]):
        for j in range(0,df.shape[1]):
            st.cell(i+sht_rn+1,j+1).value = df.iloc[i,j]

In [ ]:
# 执行写入
toexlsx(safxsip_toxlsx,'log_云外异常')
toexlsx(safxdip_toxlsx,'log_本地异常')

In [ ]:
# 保存
fxwb.save(ribaopath+'风险log.xlsx')